# Streaming

> Components for streaming content from server-sent events (SSE)

In [1]:
#| default_exp streaming

## Imports

In [2]:
#| export
import fasthtml.common as fh
from anthrosolui.foundations import *
from fasthtml.common import Div, Script, FT

In [3]:
from fasthtml.jupyter import *
from nbdev.showdoc import *
from functools import partial

## Streaming Container

This component creates a container that will display streamed content from an SSE endpoint.

In [5]:
#| export
def StreamingContainer(
    stream_url: str,           # URL endpoint that provides the SSE stream
    initial_content: str = "", # Initial content to display before streaming begins
    cls: str = "",             # Additional classes for the container
    id: str = None,            # Optional ID for the container (auto-generated if None)
    **kwargs                   # Additional arguments for the Div container
) -> FT:
    """
    Creates a container that displays streamed content from a server-sent events (SSE) endpoint.
    
    Args:
        stream_url: URL endpoint that provides the SSE stream
        initial_content: Initial content to display before streaming begins
        cls: Additional classes for the container
        id: Optional ID for the container (auto-generated if None)
        **kwargs: Additional arguments for the Div container
    
    Returns:
        FT: A Div component that will display the streamed content
    """
    if id is None:
        id = fh.unqid()
    
    return Div(
        initial_content,
        id=id,
        cls=stringify(cls),
        **{"data-stream-url": stream_url},
        **kwargs
    )

## Usage Example

Here's an example of how to use the streaming container in a FastHTML application:

In [ ]:
# Example (not run):
"""
from anthrosolui.all import Theme, fast_app, stream_script, StreamingContainer
from fasthtml.common import Div


hdrs = Theme.blue.headers(streaming=True) 

app, rt = fast_app(hdrs=hdrs)

@rt("/stream-demo")
def get():
    # Endpoint that provides the SSE stream
    stream_url = "/api/stream?message=Hello&_=123456789"
    
    return Div(
        Div("Streaming Demo", cls="text-xl font-bold mb-4"),
        StreamingContainer(
            stream_url=stream_url,
            initial_content="Waiting for response...",
            cls="p-4 border rounded bg-gray-50"
        )
    )
"""

For the server-side part, here's an example of how to create an SSE stream endpoint:

In [ ]:
# Server-side example (not run):
"""
from starlette.responses import EventSourceResponse

def sse_message(content, event="message"):
    return f"event: {event}\ndata: {content}\n\n"

@rt("/api/stream")
async def get(message: str, req):
    async def stream_generator():
        # Example: generate 10 chunks with 0.5s delay between them
        for i in range(10):
            if await req.is_disconnected():
                break
            await asyncio.sleep(0.5)
            yield sse_message(f"Chunk {i+1}: {message}")
        yield sse_message("done", event="done")  # Signal end of stream
    
    return EventSourceResponse(stream_generator())
"""

## Exports

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()